In [1]:
import pandas as pd
import holidays
import seaborn as sns
import requests
import numpy as np
from time import sleep
from shutil import copyfile
import random
from scipy import stats

In [2]:
%matplotlib inline
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
# # To find out where the pyspark
# import findspark
# findspark.init()

# # Creating Spark Context
# from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.types import StringType, IntegerType, DecimalType, DoubleType

In [4]:
from pyspark.ml.feature import VectorAssembler,CountVectorizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
"""SimpleApp.py"""
spark = SparkSession.builder.appName("FirstSparkApp").getOrCreate()

In [36]:
df = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "true") \
    .option("mode", "DROPMALFORMED")\
    .option("inferSchema", "true")\
    .load("/home/arditto_trianggada3/Workspace/ds-ovo-test/dataset/trip_merge_4_fe_all.csv")\
#     .limit(100000)

In [37]:
df1 = df.limit(10).toPandas()

In [38]:
df1.head(5)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_longitude_bin,pickup_latitude_bin,dropoff_longitude_bin,dropoff_latitude_bin,pickup_addr_zipcode,pickup_addr_place,pickup_addr_neighborhood,pickup_addr_sublocal,pickup_addr_local,pickup_addr_administrative_area_level_2,dropoff_addr_zipcode,dropoff_addr_place,dropoff_addr_neighborhood,dropoff_addr_sublocal,dropoff_addr_local,dropoff_addr_administrative_area_level_2,pickup_to_dropoff_zipcode,pickup_to_dropoff_place,pickup_to_dropoff_neighborhood,pickup_to_dropoff_sublocal,pickup_to_dropoff_local,pickup_to_dropoff_administrative_area_level_2,pickup_month,pickup_dayofweek,pickup_hour,dropoff_month,dropoff_dayofweek,dropoff_hour,is_public_holiday,is_weekend,is_holiday,trip_night,trip_late_night,trip_dist_euclidean,trip_dist_manhattan
0,91F6EB84975BBC867E32CB113C7C2CD5,AD8751110E6292079EB10EB9481FE1A6,CMT,1,N,2013-04-04 18:47:45,2013-04-04 19:00:25,1,759,2.5,-73.95785500000001,40.76532,-73.97627299999999,40.785647999999995,CRD,11.0,1.0,0.5,2.5,0.0,15.0,-73.955,40.77,-73.975,40.79,10065.0,E 67th St,None,Manhattan,New York,New York County,10024.0,W 83rd St,None,Manhattan,New York,New York County,10065.0_10024.0,E 67th St_W 83rd St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,18,4,Thursday,19,0,0,0,1,0,0.027430827694385086,0.038745999999974856
1,C1B9DA774DC2BBC6DE27CE994E7F44A0,E1B595FD55E4C82C1E213EB17438107A,CMT,1,N,2013-04-04 17:59:50,2013-04-04 18:21:48,1,1318,3.6,-73.98288000000001,40.75499,-74.009186,40.715374,CRD,16.5,1.0,0.5,3.6,0.0,21.6,-73.98,40.755,-74.005,40.72,10036.0,Verizon Plaza,None,Manhattan,New York,New York County,10007.0,Warren St,None,Manhattan,New York,New York County,10036.0_10007.0,Verizon Plaza_Warren St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,17,4,Thursday,18,0,0,0,1,0,0.04755452756572936,0.06592199999999337
2,9BA84250355AB3FC031C9252D395BF8A,16BB0D96A0DCC853AEC7F55C8D6C71E0,CMT,1,N,2013-04-04 18:12:01,2013-04-04 18:25:24,1,799,1.9,-73.97811899999999,40.763451,-73.95566600000001,40.776641999999995,CRD,10.0,1.0,0.5,3.45,0.0,14.95,-73.975,40.765,-73.955,40.78,10019.0,7th Ave,None,Manhattan,New York,New York County,10028.0,E 81st St,None,Manhattan,New York,New York County,10019.0_10028.0,7th Ave_E 81st St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,18,4,Thursday,18,0,0,0,1,0,0.026041115375480605,0.03564399999997647
3,205A696DF62AD03C88DA8C5EC5248639,579C41EA5EC846F8B641A42F9EE3E855,CMT,1,N,2013-04-04 20:12:57,2013-04-04 20:29:55,1,1017,3.6,-74.006371,40.744755,-73.961662,40.761082,CRD,15.0,0.5,0.5,3.2,0.0,19.2,-74.005,40.745,-73.96,40.765,10011.0,W 19th St,None,Manhattan,New York,New York County,10022.0,E 60th St,None,Manhattan,New York,New York County,10011.0_10022.0,W 19th St_E 60th St,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Thursday,20,4,Thursday,20,0,0,0,1,1,0.04759690756761306,0.06103600000000142
4,EE75E5927D00739AC342810C336A825E,1B4E92431F9DA4D49874EC76E769E874,CMT,1,N,2013-04-05 02:48:11,2013-04-05 02:51:21,2,189,0.7,-73.985191,40.754932000000004,-73.990776,40.747997,CRD,4.5,0.5,0.5,1.1,0.0,6.6,-73.985,40.755,-73.99,40.75,10036.0,Verizon Plaza,None,Manhattan,New York,New York County,10120.0,West 33rd Street,None,Manhattan,New York,New York County,10036.0_10120.0,Verizon Plaza_West 33rd Street,nan_nan,Manhattan_Manhattan,New York_New York,New York County_New York County,4,Friday,2,4,Friday,2,0,0,0,0,1,0.008904293907999591,0.012520000000002085


In [39]:
df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- pickup_longitude_bin: string (nullable = true)
 

In [40]:
df = df\
    .withColumn("passenger_count", F.col("passenger_count").cast(IntegerType()))\
    .withColumn("trip_time_in_secs", F.col("trip_time_in_secs").cast(IntegerType()))\
    .withColumn("trip_distance", F.col("trip_distance").cast(DoubleType()))\
    .withColumn("fare_amount", F.col("fare_amount").cast(DoubleType()))\
    .withColumn("tip_amount", F.col("fare_amount").cast(DoubleType()))\
    .withColumn("is_public_holiday", F.col("is_public_holiday").cast(IntegerType()))\
    .withColumn("is_weekend", F.col("is_weekend").cast(IntegerType()))\
    .withColumn("is_holiday", F.col("is_holiday").cast(IntegerType()))\
    .withColumn("trip_night", F.col("trip_night").cast(IntegerType()))\
    .withColumn("trip_late_night", F.col("trip_late_night").cast(IntegerType()))\
    .withColumn("trip_dist_euclidean", F.col("trip_dist_euclidean").cast(DoubleType()))\
    .withColumn("trip_dist_manhattan", F.col("trip_dist_manhattan").cast(DoubleType()))

In [41]:
feat_cat = [
    'pickup_addr_zipcode','pickup_addr_local',
    'dropoff_addr_zipcode','dropoff_addr_local',
    'pickup_to_dropoff_zipcode','pickup_to_dropoff_local',
    'pickup_dayofweek','pickup_hour',
    'dropoff_dayofweek','dropoff_hour',
] 

In [33]:
for c in feat_cat:
    df = df.withColumn(c, F.split(F.col(c),"  "))

In [35]:
catVectorizer = {}
catVectorizer_model = {}

for c in feat_cat:
    # Initialize a CountVectorizer.
    catVectorizer[c] = CountVectorizer(inputCol=c, outputCol="{}".format(c), minDF=1.0)
    #Get a VectorizerModel
    catVectorizer_model[c] = catVectorizer[c].fit(df)

In [48]:
for c in feat_cat:
    df = catVectorizer_model[c].transform(df)

In [42]:
feat_excl = [
    'medallion','hack_license','vendor_id',
    'rate_code','store_and_fwd_flag',
    'pickup_datetime','dropoff_datetime',
    'pickup_month','dropoff_month',
    'pickup_longitude','pickup_latitude',
    'dropoff_longitude','dropoff_latitude',
    'surcharge','mta_tax','tip_amount','tolls_amount','payment_type','total_amount',
    'pickup_addr_neighborhood','pickup_addr_sublocal','pickup_addr_place','pickup_addr_administrative_area_level_2',
    'dropoff_addr_neighborhood','dropoff_addr_sublocal','dropoff_addr_place','dropoff_addr_administrative_area_level_2',
    'pickup_to_dropoff_neighborhood','pickup_to_dropoff_sublocal','pickup_to_dropoff_place','pickup_to_dropoff_administrative_area_level_2',
    'pickup_longitude_bin','pickup_latitude_bin',
    'dropoff_longitude_bin','dropoff_latitude_bin'
]

In [43]:
# feat_excl = feat_excl + feat_cat

In [44]:
vectorAssembler = VectorAssembler(inputCols = [c for c in df.columns if not(c in feat_excl)], outputCol = 'features')

df = vectorAssembler.transform(df)

In [45]:
df = df\
    .withColumnRenamed("tip_amount","label")\
    .select("features","label")

In [46]:
df.columns

['features', 'label']

In [47]:
splits = df.randomSplit([0.7, 0.3])

train_df = splits[0]
test_df = splits[1]

In [48]:
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [49]:
lr_model = lr.fit(train_df)

In [50]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.001571323145213063,0.4589487477550978,0.7669993515534895,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 0.3449105187158554


In [51]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.587602
r2: 0.995682


In [52]:
train_df.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|             70073|
|   mean|12.146971729482111|
| stddev| 8.941834923350573|
|    min|               2.5|
|    max|             141.0|
+-------+------------------+



In [53]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","label","features").show(5)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|12.056387443371323| 12.0|(11,[0,1,2,3],[1....|
| 3.311856624445442|  3.5|(11,[0,1,2,3,5,6]...|
| 3.362340986698503|  3.5|(11,[0,1,2,3,5,6]...|
| 3.745840662475248|  4.0|(11,[0,1,2,3,5,6]...|
| 3.775867226502318|  4.0|(11,[0,1,2,3,5,6]...|
+------------------+-----+--------------------+
only showing top 5 rows



In [54]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")

print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R Squared (R2) on test data = 0.995364


In [55]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 0.611337


In [56]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 11
objectiveHistory: [0.5000000000000002, 0.40807471477948365, 0.05463866088039461, 0.0534492334305488, 0.037042490880576706, 0.03505597965869335, 0.03479696056842353, 0.034378827329315687, 0.03392093624396905, 0.0326865590624646, 0.031277737110982334]
+--------------------+
|           residuals|
+--------------------+
|  11.653896374402144|
|-0.07593901015395588|
| -0.3413643604617391|
|  0.1404853634945673|
| 0.11753792610681213|
| 0.22485927537496542|
| 0.16978542564435362|
| 0.12848003834639465|
| 0.17647476143769136|
| 0.08009552440912104|
| 0.04337962458871303|
| 0.14152199890176442|
| 0.11398507403645874|
| 0.18459052348420357|
| 0.13869564870869322|
| 0.10231645991959226|
| 0.08854799748693942|
| 0.08395851000938848|
| 0.02429517280122573|
|  0.1086690846816234|
+--------------------+
only showing top 20 rows



In [57]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","label","features").show()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|12.056387443371323| 12.0|(11,[0,1,2,3],[1....|
| 3.311856624445442|  3.5|(11,[0,1,2,3,5,6]...|
| 3.362340986698503|  3.5|(11,[0,1,2,3,5,6]...|
| 3.745840662475248|  4.0|(11,[0,1,2,3,5,6]...|
| 3.775867226502318|  4.0|(11,[0,1,2,3,5,6]...|
| 3.415314988113328|  3.5|(11,[0,1,2,3,5,6]...|
| 3.429083450545981|  3.5|(11,[0,1,2,3,5,6]...|
|3.8814254384859903|  4.0|(11,[0,1,2,3,5,6]...|
|  4.28328306417294|  4.5|(11,[0,1,2,3,5,6]...|
|3.9206309774681625|  4.0|(11,[0,1,2,3,5,6]...|
| 4.907925727944099|  5.0|(11,[0,1,2,3,5,6]...|
| 4.944641627764508|  5.0|(11,[0,1,2,3,5,6]...|
| 5.346499253451456|  5.5|(11,[0,1,2,3,5,6]...|
| 4.471841841598507|  4.5|(11,[0,1,2,3,5,6]...|
| 5.339809917658118|  5.5|(11,[0,1,2,3,5,6]...|
| 5.482084029462198|  5.5|(11,[0,1,2,3,5,6]...|
| 6.012447304520575|  6.0|(11,[0,1,2,3,5,6]...|
| 4.899136543834975|  5.0|(11,[0,1,2,3,5